In [ ]:
%matplotlib inline
from pprint import pprint
from decimal import Decimal
import itertools
import metrics
import pandas

import simulate
import harvesting
import market
import withdrawal
import numpy

In [ ]:
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
series = market.Returns_US_1871()

In [ ]:
TYPE = 'Withdraw Pct Cur'
m = {'Withdrawal (Nominal)': lambda x: float(x.withdraw_n),
     'Withdrawal (Real)' : lambda x: float(x.withdraw_r),
     'Portfolio (Nominal)': lambda x: float(x.portfolio_n),
     'Portfolio (Real)': lambda x: float(x.portfolio_r),
     'Withdraw Pct Cur': lambda x: float(x.withdraw_pct_cur),
}
lens = m[TYPE]
lens.__label__ = TYPE

def fn(x):
    return [lens(_) for _ in x]

In [ ]:
def run_sim(year, length=30):
    portfolio = (500000,500000)
    harvest_strategy = harvesting.make_rebalancer(0.5)
    
    return simulate.withdrawals(series.iter_from(year),
                                 withdraw=lambda p, s: withdrawal.ConstantDollar(p, s, rate=Decimal('0.0325')),
                                 years=length,
                                 portfolio=portfolio,
                                 harvesting=harvest_strategy)

In [ ]:
def chart_one(year):
    s = run_sim(year)
    
    df = pandas.DataFrame({
        'pmt' : [lens(x) for x in s],
        'cape' : 0.02,
    })
    g = seaborn.relplot(
            data=df,
            kind='line',
            aspect=2,
        )
    g.fig.autofmt_xdate()
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle(f'Retirement in {year}')
    return g

In [ ]:
all_sims = {}
for year in range(1871, 1989):
    all_sims[year] = [lens(x) for x in run_sim(year)]

In [ ]:
df = pandas.DataFrame(all_sims)

In [ ]:
s = df[df <= 0.02].count()
s.plot.line()

In [ ]:
df.T.loc[1945].plot.line()

In [ ]:
s[s > 0].median()

In [ ]:
divs = pandas.Series(index=range(len(df)))
for i in range(len(df)):
    s = df.loc[i]
    d = s[s <= 0.02]
    divs.loc[i] = len(d) / len(s)
#    print(i, len(d), len(s))
divs.plot.line()

In [ ]:
divs